In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
src_dataset = np.load(file='user_source.npy', allow_pickle=True)
tar_dataset = np.load(file='user_target.npy', allow_pickle=True)

print(len(src_dataset))
print(len(tar_dataset))

426
426


In [3]:
src_df = pd.DataFrame({'src':[]})
for i in range(len(src_dataset)) :
    src_df.loc[i] = [src_dataset[i]]

tar_df = pd.DataFrame({'tar':[]})
for i in range(len(tar_dataset)) :
    tar_df.loc[i] = [tar_dataset[i]]

df = pd.concat([src_df, tar_df], axis=1)
df.head()

,src,tar
0,"[[39.9844254, 116.3101288], [39.9841684, 116.2...","[[39.984702, 116.318417], [39.984683, 116.3184..."
1,"[[39.990832, 116.309828], [39.988258, 116.2933...","[[39.995777, 116.286798], [39.996832, 116.2854..."
2,"[[40.0087849, 116.3218099], [40.0089858, 116.3...","[[40.008921, 116.321484], [40.008582, 116.3222..."
3,"[[40.00815, 116.3204629], [40.0092487, 116.318...","[[40.009351, 116.321916], [40.009336, 116.3218..."
4,"[[40.00815, 116.3204629], [40.0088233, 116.321...","[[40.006747, 116.318549], [40.006747, 116.3185..."


In [4]:
# <SOS> [0, 0], <EOS> [-1, -1]
for tar in df.tar :
    tar.insert(0, [0, 0])
    tar.insert(len(tar), [-1, -1])
df.head()

,src,tar
0,"[[39.9844254, 116.3101288], [39.9841684, 116.2...","[[0, 0], [39.984702, 116.318417], [39.984683, ..."
1,"[[39.990832, 116.309828], [39.988258, 116.2933...","[[0, 0], [39.995777, 116.286798], [39.996832, ..."
2,"[[40.0087849, 116.3218099], [40.0089858, 116.3...","[[0, 0], [40.008921, 116.321484], [40.008582, ..."
3,"[[40.00815, 116.3204629], [40.0092487, 116.318...","[[0, 0], [40.009351, 116.321916], [40.009336, ..."
4,"[[40.00815, 116.3204629], [40.0088233, 116.321...","[[0, 0], [40.006747, 116.318549], [40.006747, ..."


In [5]:
encoder_input = []
for sequence in df.src :
    encoder_input.append(sequence)

decoder_input = []
for sequence in df.tar :
    decoder_input.append(sequence[:-1])

decoder_target = []
for sequence in df.tar :
    decoder_target.append(sequence[1:])

In [6]:
# padding
encoder_input = pad_sequences(encoder_input, padding='post')
decoder_input = pad_sequences(decoder_input, padding='post')
decoder_target = pad_sequences(decoder_target, padding='post')

In [7]:
# 426 samples, length of src is 14934, length of tar is 1490
print(np.shape(encoder_input))
print(np.shape(decoder_input))
print(np.shape(decoder_target))

(426, 14934, 2)
(426, 1490, 2)
(426, 1490, 2)
